In [1]:
# Packages

# To make data frames
import pandas as pd

# Numerical calculations
import numpy as np

# Timer
import time # for debugging

# For date arithmetic
from datetime import datetime, timedelta

In [2]:
# Functions

# Get how long a function takes (needs time)
def time_elapsed(start, end):
    total_time_seconds = end-start
    print("Total time elapsed:") 
    if total_time_seconds < 60:
        print(total_time_seconds, "seconds")
    else:
        total_time_minutes = total_time_seconds/60 
        if total_time_minutes < 60: 
            print(total_time_minutes, "minutes") 
        else: 
            total_time_hours = total_time_minutes/60 
            print(total_time_hours, "hours")
    print() 
    
# Add a row to df with given end_date, candidate_name, and party (needs df)
def insert_row(date, candidate, pop):
    global df
    df.at[len(df), 'end_date'] = date
    #df['end_date'][len(df-1)] = pd.to_datetime(df['end_date'][len(df-1)]).dt.date()
    df.at[len(df)-1, 'candidate_name'] = candidate
    df.at[len(df)-1, 'population'] = pop

# Impute a missing date for a candidate (needs the df)
def impute_for_candidate(date, candidate, pop):
    print("date, candidate, pop are "+str(date)+", "+candidate+", "+pop) # for debugging
    if ((df['end_date'] == date) & (df['candidate_name'] == candidate) & (df['population'] == pop)).any() == False:   
        print("No entry for this date, candidate, population.") # for debugging
        insert_row(date, candidate, pop)    
        print("Now we've inserted a row.") # for debugging
    else: # for debugging
        print("Candidate has an entry here.") # for debugging
    print() # for debugging

In [3]:
# Read in the data

old_data = pd.read_csv("president_polls_cleaned.csv")
updated_data = pd.read_csv("president_polls.csv")

# Parse the dates

old_data['end_date'] = pd.to_datetime(old_data['end_date'], format='mixed').dt.date
updated_data['end_date'] = pd.to_datetime(updated_data['end_date'], format='mixed').dt.date

C:\Users\leyjf\AppData\Local\Temp\ipykernel_22660\83704689.py:4: DtypeWarning: Columns (16,17,26) have mixed types. Specify dtype option on import or set low_memory=False.
  updated_data = pd.read_csv("president_polls.csv")


In [4]:
# Get the new data, along with the last week of old data

last_date = old_data['end_date'][0]

In [5]:
df = updated_data[updated_data['end_date'] > last_date]
df['end_date']

0     2024-10-29
1     2024-10-29
2     2024-10-29
3     2024-10-29
4     2024-10-29
         ...    
64    2024-10-29
65    2024-10-29
66    2024-10-29
67    2024-10-29
68    2024-10-29
Name: end_date, Length: 69, dtype: object

In [6]:
# Restrict to national and swing state polls

swing_states = ["Arizona", "Georgia", "Michigan", "Nevada", "North Carolina", "Pennsylvania", "Wisconsin"]
df = df.loc[df['state'].isin(swing_states) | df['state'].isnull()]

In [7]:
# Delete superfluous columns

df.drop(columns=['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'display_name', 'pollster_rating_id', 
                 'sponsor_candidate_id', 'sponsor_candidate', 'sponsor_candidate_party', 'endorsed_candidate_id', 
                 'endorsed_candidate_name', 'endorsed_candidate_party', 'subpopulation', 'population_full', 'tracking', 
                 'created_at', 'notes', 'url', 'source', 'internal', 'race_id', 'cycle', 'office_type', 'seat_number', 
                 'seat_name', 'election_date', 'stage', 'nationwide_batch', 'ranked_choice_reallocated', 
                 'ranked_choice_round', 'answer', 'candidate_id'], inplace=True)

In [8]:
# Replace the 'v' in the population column with 'rv'

df['population'].replace('v', 'rv', inplace=True)

C:\Users\leyjf\AppData\Local\Temp\ipykernel_22660\727013223.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['population'].replace('v', 'rv', inplace=True)


In [9]:
# Use groupby to get daily averages according to population polled

cand_avg_by_date = df.groupby(['end_date', 'candidate_name', 'population'])['pct'].mean().reset_index(name='da_by_pop')
df = pd.merge(df, cand_avg_by_date)

In [10]:
# Get lists for imputing 

all_dates = df['end_date'].drop_duplicates() #pd.date_range(df['end_date'].tolist()[-1], df['end_date'].tolist()[0], freq='D')
candidates = ['Donald Trump', 'Joe Biden', 'Kamala Harris'] 
pops = ['lv', 'rv', 'a']

In [11]:
all_dates

0    2024-10-29
Name: end_date, dtype: object

In [12]:
# Impute the missing dates, candidate, populations 

start_impute = time.time()
for date in all_dates:
    for candidate in candidates:
        for pop in pops:
            impute_for_candidate(date, candidate, pop)
end_impute = time.time()
time_elapsed(start_impute, end_impute)   

date, candidate, pop are 2024-10-29, Donald Trump, lv
Candidate has an entry here.

date, candidate, pop are 2024-10-29, Donald Trump, rv
Candidate has an entry here.

date, candidate, pop are 2024-10-29, Donald Trump, a
No entry for this date, candidate, population.
Now we've inserted a row.

date, candidate, pop are 2024-10-29, Joe Biden, lv
No entry for this date, candidate, population.
Now we've inserted a row.

date, candidate, pop are 2024-10-29, Joe Biden, rv
No entry for this date, candidate, population.
Now we've inserted a row.

date, candidate, pop are 2024-10-29, Joe Biden, a
No entry for this date, candidate, population.
Now we've inserted a row.

date, candidate, pop are 2024-10-29, Kamala Harris, lv
Candidate has an entry here.

date, candidate, pop are 2024-10-29, Kamala Harris, rv
Candidate has an entry here.

date, candidate, pop are 2024-10-29, Kamala Harris, a
No entry for this date, candidate, population.
Now we've inserted a row.

Total time elapsed:
0.00943899154

In [13]:
# Reparse the dates, then take only the crucial data

#df['end_date'] = pd.to_datetime(df['end_date'], format='mixed').dt.date
crucial_data = df.loc[(df['candidate_name'] == 'Donald Trump') | (df['candidate_name'] == 'Joe Biden') | 
       (df['candidate_name'] == 'Kamala Harris'), ['population', 'candidate_name', 
                                                   'end_date', 'da_by_pop']].drop_duplicates()


In [14]:
crucial_data.sort_values(['end_date', 'candidate_name', 'population']).tail(50)

,population,candidate_name,end_date,da_by_pop
63,a,Donald Trump,2024-10-29,NaN
1,lv,Donald Trump,2024-10-29,49.025
9,rv,Donald Trump,2024-10-29,46.000
66,a,Joe Biden,2024-10-29,NaN
64,lv,Joe Biden,2024-10-29,NaN
65,rv,Joe Biden,2024-10-29,NaN
67,a,Kamala Harris,2024-10-29,NaN
0,lv,Kamala Harris,2024-10-29,47.815
8,rv,Kamala Harris,2024-10-29,47.000


In [15]:
# Add in the last six days of the old data for the rolling average

crucial_data = pd.concat([old_data[old_data['end_date'] > last_date-pd.Timedelta(days=7)], crucial_data], ignore_index=True)
crucial_data

,population,candidate_name,end_date,da_by_pop,pop_raverage7
0,Registered voters,Donald Trump,2024-10-28,50.00,47.838889
1,Registered voters,Donald Trump,2024-10-27,50.00,46.946032
2,Registered voters,Donald Trump,2024-10-26,50.00,46.437037
3,Registered voters,Donald Trump,2024-10-25,45.55,45.724444
4,Registered voters,Donald Trump,2024-10-24,45.55,45.768056
...,...,...,...,...,...
67,a,Donald Trump,2024-10-29,NaN,NaN
68,lv,Joe Biden,2024-10-29,NaN,NaN
69,rv,Joe Biden,2024-10-29,NaN,NaN
70,a,Joe Biden,2024-10-29,NaN,NaN


In [16]:
# Format populations

crucial_data['population'].replace(['a', 'rv', 'lv'], ['All adults', ' Registered voters', 'Likely voters'], inplace=True)
crucial_data

C:\Users\leyjf\AppData\Local\Temp\ipykernel_22660\1771588769.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  crucial_data['population'].replace(['a', 'rv', 'lv'], ['All adults', ' Registered voters', 'Likely voters'], inplace=True)


,population,candidate_name,end_date,da_by_pop,pop_raverage7
0,Registered voters,Donald Trump,2024-10-28,50.00,47.838889
1,Registered voters,Donald Trump,2024-10-27,50.00,46.946032
2,Registered voters,Donald Trump,2024-10-26,50.00,46.437037
3,Registered voters,Donald Trump,2024-10-25,45.55,45.724444
4,Registered voters,Donald Trump,2024-10-24,45.55,45.768056
...,...,...,...,...,...
67,All adults,Donald Trump,2024-10-29,NaN,NaN
68,Likely voters,Joe Biden,2024-10-29,NaN,NaN
69,Registered voters,Joe Biden,2024-10-29,NaN,NaN
70,All adults,Joe Biden,2024-10-29,NaN,NaN


In [17]:
# Sort, then format the empty entries
 
crucial_data.sort_values(['candidate_name', 'population', 'end_date'], inplace=True, ignore_index=True)
crucial_data = crucial_data.replace('', np.NaN)
crucial_data.head(50)

,population,candidate_name,end_date,da_by_pop,pop_raverage7
0,Registered voters,Donald Trump,2024-10-22,48.222222,45.986111
1,Registered voters,Donald Trump,2024-10-23,45.550000,45.840741
2,Registered voters,Donald Trump,2024-10-24,45.550000,45.768056
3,Registered voters,Donald Trump,2024-10-25,45.550000,45.724444
4,Registered voters,Donald Trump,2024-10-26,50.000000,46.437037
5,Registered voters,Donald Trump,2024-10-27,50.000000,46.946032
6,Registered voters,Donald Trump,2024-10-28,50.000000,47.838889
7,Registered voters,Donald Trump,2024-10-29,46.000000,NaN
8,All adults,Donald Trump,2024-10-22,47.000000,43.000000
9,All adults,Donald Trump,2024-10-23,43.050000,43.016667


In [18]:
# Forward fill the missing averages
 
crucial_data['da_by_pop'] = crucial_data.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.ffill())
crucial_data.head(50)

,population,candidate_name,end_date,da_by_pop,pop_raverage7
0,Registered voters,Donald Trump,2024-10-22,48.222222,45.986111
1,Registered voters,Donald Trump,2024-10-23,45.550000,45.840741
2,Registered voters,Donald Trump,2024-10-24,45.550000,45.768056
3,Registered voters,Donald Trump,2024-10-25,45.550000,45.724444
4,Registered voters,Donald Trump,2024-10-26,50.000000,46.437037
5,Registered voters,Donald Trump,2024-10-27,50.000000,46.946032
6,Registered voters,Donald Trump,2024-10-28,50.000000,47.838889
7,Registered voters,Donald Trump,2024-10-29,46.000000,NaN
8,All adults,Donald Trump,2024-10-22,47.000000,43.000000
9,All adults,Donald Trump,2024-10-23,43.050000,43.016667


In [19]:
# Add column for rolling averages
   
crucial_data['pop_raverage7'] = crucial_data.groupby(['candidate_name', 
        'population']).rolling(7, min_periods=1)['da_by_pop'].mean().reset_index(drop=True)  

In [20]:
crucial_data.head(50)

,population,candidate_name,end_date,da_by_pop,pop_raverage7
0,Registered voters,Donald Trump,2024-10-22,48.222222,48.222222
1,Registered voters,Donald Trump,2024-10-23,45.550000,46.886111
2,Registered voters,Donald Trump,2024-10-24,45.550000,46.440741
3,Registered voters,Donald Trump,2024-10-25,45.550000,46.218056
4,Registered voters,Donald Trump,2024-10-26,50.000000,46.974444
5,Registered voters,Donald Trump,2024-10-27,50.000000,47.478704
6,Registered voters,Donald Trump,2024-10-28,50.000000,47.838889
7,Registered voters,Donald Trump,2024-10-29,46.000000,47.521429
8,All adults,Donald Trump,2024-10-22,47.000000,47.000000
9,All adults,Donald Trump,2024-10-23,43.050000,45.025000


In [21]:
# Sort again

crucial_data.sort_values(by=['candidate_name', 'population', 'end_date'], ascending=[True, True, False], inplace=True)

In [22]:
# Remove the last six days from the old_data

old_data = old_data[old_data['end_date'] <= last_date-pd.Timedelta(days=7)]
old_data

,population,candidate_name,end_date,da_by_pop,pop_raverage7
7,Registered voters,Donald Trump,2024-10-21,43.750000,43.750000
8,Registered voters,Donald Trump,2024-10-20,47.294118,46.937255
9,Registered voters,Donald Trump,2024-10-19,48.000000,46.800000
10,Registered voters,Donald Trump,2024-10-18,45.866667,46.728571
11,Registered voters,Donald Trump,2024-10-17,45.866667,46.961905
...,...,...,...,...,...
11704,Likely voters,Kamala Harris,2021-04-11,NaN,NaN
11705,Likely voters,Kamala Harris,2021-04-10,NaN,NaN
11706,Likely voters,Kamala Harris,2021-04-09,NaN,NaN
11707,Likely voters,Kamala Harris,2021-04-08,NaN,NaN


In [23]:
# Combine new data with old data, then export to a file

all_data = pd.concat([old_data, crucial_data], ignore_index=True)
all_data.sort_values(by=['candidate_name', 'population', 'end_date'], ascending=[True, True, False], inplace=True)
all_data.to_csv("president_polls_cleaned.csv", index=False)